# Starting data analysis

In [180]:
import pandas as pd
basal_52104 = pd.read_csv("Data/PatientData/EAOB/52104/basal.csv")
bolus_52104 = pd.read_csv("Data/PatientData/EAOB/52104/bolus.csv")
cgm_52104 = pd.read_csv("Data/PatientData/EAOB/52104/cgm.csv")
hr_52104 = pd.read_csv("Data/PatientData/EAOB/52104/hr.csv")
meal_52104 = pd.read_csv("Data/PatientData/EAOB/52104/meal.csv")
dataFrames = [basal_52104, bolus_52104, cgm_52104] # , meal_52104

# for frame in dataFrames:
#     print(frame.columns)

In [181]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def refactorTime(df):
    name = get_df_name(df).split("_")[0]

    try:
        df.iloc[:, 1] = pd.to_datetime(df.iloc[:, 1], format="%m/%d/%Y %H:%M") #02/27/2018 00:01
    except:
        df.iloc[:, 1] = pd.to_datetime(df.iloc[:, 1], format="%m/%d/%Y %H:%M:%S") #02/27/2018 00:01:08
    
    df.iloc[:, 2] = df.iloc[:, 2].apply(pd.to_numeric, errors='coerce')

    # set index to time, this makes df a time series df and then apply pandas time series functions.  
    df.set_index(df.iloc[:, 1], drop=True, inplace=True)   

    # create another df by resampling the original df
    new_df = pd.DataFrame(df.iloc[:, 2].resample("5min", label="right", closed="right").mean().pad())
    new_df = new_df.rename(columns={" Reading": name + " reading", " Units": name + " reading"})
    new_df = new_df.rename_axis("time")
    return new_df

dataFrames_1 = []
for frame in dataFrames:
    frame = refactorTime(frame)
    # print(frame.head())
    dataFrames_1.append(frame)
df_52104 = pd.concat(dataFrames_1, axis=1)
# print(df_52104.head())
df_52104.to_csv("df_52104.csv")